In [1]:
# library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

C:\Users\hp\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# here we load documents and query 

with open("dataset/docs.json", "r", encoding="utf-8") as doc, open("dataset/queries_train.json", "r", encoding="utf-8") as query:

    documents = json.load(doc)
    queries = json.load(query)

df_doc = pd.DataFrame(documents)
df_query = pd.DataFrame(queries)

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 417.65it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
# Fusion de title, text et tags en une seule colonne "content"
def merge_content(row):
    title = row.get("title") or ""      # vide si None
    text = row.get("text") or ""
    tags = " ".join(row.get("tags", []))  # transformer la liste en chaîne
    
    return f"{title} {text} {tags}".strip()

In [8]:
df_doc["content"] = df_doc.apply(merge_content, axis=1)
df_query["content"] = df_query.apply(merge_content, axis=1)

doc_texts = df_doc["content"].tolist()
query_texts = df_query["content"].tolist()

In [ ]:
doc_embeddings = model.encode(doc_texts, convert_to_numpy=True)
query_embeddings = model.encode(query_texts, convert_to_numpy=True)

In [ ]:
similarity_matrix = cosine_similarity(query_embeddings, doc_embeddings)

In [ ]:
k = 5
top_k_indices = np.argsort(-similarity_matrix, axis=1)[:, :k]

In [ ]:
embedding_results = {}

for i, query_id in enumerate(df_query["id"]):
    retrieved_docs = df_doc.iloc[top_k_indices[i]]["id"].tolist()
    embedding_results[query_id] = retrieved_docs

In [ ]:
result = pd.DataFrame(embedding_results)
result